In [ ]:
!pip install --upgrade pip
!pip install pydot
!pip install keras

In [ ]:
import os
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.xkcd()

from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Embedding, Reshape, Activation, Dropout, concatenate
from keras.models import Model

from tensorflow.keras.models import Sequential
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform, he_uniform

from tensorflow.keras.layers import Layer, InputSpec

from keras.regularizers import l2, l1_l2
from keras import backend as K
from keras.utils.vis_utils import plot_model

import keras
import pydot as pyd

from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
df_rating = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u1.base', sep = '\t', engine='python', header=None)
df_rating.columns = ['UserId', 'MovieId', 'Rating', 'Timestamp']
df_rating_test = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u1.test', sep = '\t', engine='python', header=None)

df_users = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.user', sep = '|', engine='python', header=None)
df_users.columns = ['UserId', 'Age', 'Gender', 'Occupation', 'ZipCode']
df_users.set_index('UserId', inplace = True)
df_items = pd.read_csv('http://files.grouplens.org/datasets/movielens/ml-100k/u.item', sep = '|', engine='python', encoding ='ISO-8859-1', header=None)
df_items.columns = ['MovieId', 'Title', 'Date', 'VideoReleaseDate', 'Url', 'unknown', 'Action','Adventure', 'Animation', 
                    'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror','Musical', 'Mystery', 'Romance', 
                    'Sci-Fi', 'Thriller', 'War', 'Western']
df_items.set_index('MovieId', inplace = True)

df_matrix = df_rating.pivot(index='UserId', columns='MovieId', values='Rating')

n_users = len(df_users)
n_items = len(df_items)

In [ ]:
df_matrix

In [ ]:
df_users

In [ ]:
df_items

In [ ]:
def get_pos(df, anchor):
    POS_THR = 4.0
    
    ps = df.loc[anchor] >= POS_THR 
    return ps[ps].index.values

In [ ]:
def get_neg(df, anchor):
    NEG_THR = 3.0
    
    ps = df.loc[anchor] <= NEG_THR 
    return ps[ps].index.values

In [ ]:
user_con_features = ["Age"]
user_cat_features = ["Gender", "Occupation"]

item_features = ['Action','Adventure', 'Animation', 
                'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 
                'Fantasy','Film-Noir', 'Horror','Musical', 'Mystery', 'Romance', 
                'Sci-Fi', 'Thriller', 'War', 'Western']

In [ ]:
df_users

In [ ]:
def process_features(df, con_features, cat_features):
    # initialise an empty lists of vectors
    X_cat = []
    X_con = []

    # for each continuous feature, add it as it is.
    for column in con_features:
        X = np.array(df[column])
        X_con.append(X)

    # for each categorical feature, get the numerical encoding of the feature vector
    for column in cat_features:
        X = np.asarray(df[column].tolist())
        X_line = pd.factorize(X)[0]
        X_cat.append(np.asarray(X_line))
    
    # transform lists in arrays
    try:
        X_con = np.column_stack(X_con)
    except:
        pass

    try:    
        X_cat = np.column_stack(X_cat)
    except:
        pass

    # concatenate arrays
    if (len(cat_features) > 0) and (len(con_features) > 0): 
        X = np.concatenate((X_con, X_cat), axis = 1)
    elif len(cat_features) == 0:
        X = X_con
    elif len(con_features) == 0:
        X = X_cat
    # return the encoded features
    return X

In [ ]:
X_usr = process_features(df_users, user_con_features, user_cat_features)
X_item = process_features(df_items, item_features, [])

In [ ]:
class TripletLossLayer(Layer):
    """
        Layer object to minimise the triplet loss.
        Here we implement the Bayesian Personal Ranking triplet loss.
    """
    def __init__(self, **kwargs):
        super(TripletLossLayer, self).__init__(**kwargs)
    
    def bpr_triplet_loss(self, inputs):
        """
            Bayesian Personal Ranking triplet loss.
            We make use of log-loss for numerical purposes.
        """
        anchor, positive, negative = inputs
        p_score = K.dot(anchor,K.transpose(positive))
        n_score = K.dot(anchor,K.transpose(negative))
        return K.log(1.0 - K.sigmoid(p_score - n_score))
    
    def call(self, inputs):
        loss = self.bpr_triplet_loss(inputs)
        self.add_loss(loss)
        return loss

class ScoreLayer(Layer):
    """
        Layer object to predict positive matches.
    """
    def __init__(self, **kwargs):
        super(ScoreLayer, self).__init__(**kwargs)
    
    def rec_similarity(self, inputs):
        """
            rec_similarity function
        """
        anchor, item = inputs
        score = K.dot(anchor,K.transpose(item))
        return score
    
    def call(self, inputs):
        pred = self.rec_similarity(inputs)
        return pred

In [ ]:
def build_model(n_users, n_items, emb_dim = 30):
    n_user_features = 3
    n_item_features = 18

    ### Input Layers

    user_input = Input((n_user_features,), name='user_input')
    positive_item_input = Input((n_item_features,), name='pos_item_input')
    negative_item_input = Input((n_item_features,), name='neg_item_input')

    inputs = [user_input, positive_item_input, negative_item_input]

    ### Embedding Layers

    user_emb = Embedding(n_users, emb_dim, input_length=n_user_features, name='user_emb')
    # Positive and negative items will share the same embedding
    item_emb = Embedding(n_items, emb_dim, input_length=n_item_features, name='item_emb')
    # Layer to convert embedding vectors in the same dimensional vectors
    vec_conv64 = Dense(64, name = 'dense_vec64', activation = 'relu')
    vec_conv32 = Dense(32, name = 'dense_vec32', activation = 'relu')
    vec_conv = Dense(emb_dim, name = 'dense_vec', activation = 'softmax')
    

    # Anchor
    a = Flatten(name = 'flatten_usr_emb')(user_emb(user_input))
    a = Dense(emb_dim, name = 'dense_user', activation = 'softmax')(a)
    
    # Positive
    p = Flatten(name = 'flatten_pos_emb')(item_emb(positive_item_input))
    #p = vec_conv64(p)
    p = vec_conv32(p)
    p = vec_conv(p)

    # Negative
    n = Flatten(name = 'flatten_neg_emb')(item_emb(negative_item_input))
    #n = vec_conv64(n)
    n = vec_conv32(n)
    n = vec_conv(n)

    #Force the encoding to live on the d-dimentional hypershpere
    #a = Lambda(lambda x: K.l2_normalize(x), name = 'normalise_layer_a')(a)
    #p = Lambda(lambda x: K.l2_normalize(x), name = 'normalise_layer_p')(p)
    #n = Lambda(lambda x: K.l2_normalize(x), name = 'normalise_layer_n')(n)

    # Score layers
    p_rec_score = ScoreLayer(name='pos_recommendation_score')([a, p])
    n_rec_score = ScoreLayer(name='neg_recommendation_score')([a, n])
    
    # TripletLoss Layer
    loss_layer = TripletLossLayer(name='triplet_loss_layer')([a, p, n])
    
    # Connect the inputs with the outputs
    network_train = Model(inputs=inputs, outputs=loss_layer, name = 'training_model')

    network_predict = Model(inputs=inputs[:-1], outputs=p_rec_score, name = 'inference_model')

    # return the model
    return network_train, network_predict

In [ ]:
network_train, network_predict = build_model(n_users, n_items)
optimizer = Adam(lr = 0.001)
network_train.compile(loss=None,optimizer=optimizer)
network_train.summary()
#plot_model(network_train,show_shapes=True, show_layer_names=True, to_file='model.png')
n_iteration=0

In [ ]:
network_predict.layers

In [ ]:
network_train.layers

In [ ]:
 #Training phase
    def cosine_dist(x,y, eps = 1e-6):
    return 1 - np.dot(x,y.T)/(np.linalg.norm(x)*np.linalg.norm(y) + eps)

In [ ]:
def build_batch(batch_size, X_usr, X_item, df, return_cache = False):
            
    # constant values
    n_user_features = X_usr.shape[1]
    n_item_features = X_item.shape[1]

    # define user_list
    user_list = list(df.index.values)

    # initialise result
    triplets = [np.zeros((batch_size, n_user_features)), # anchor
                np.zeros((batch_size, n_item_features)), # pos
                np.zeros((batch_size, n_item_features))  # neg
                ]
    user_ids = []
    p_ids = []
    n_ids = []

    for i in range(batch_size):
        # pick one random user for anchor
        anchor_id = random.choice(user_list)
        user_ids.append(anchor_id) 

        # all possible positive/negative samples for selected anchor
        p_item_ids = get_pos(df, anchor_id)
        n_item_ids = get_neg(df, anchor_id)

        # pick one of the positve ids
        try:
            positive_id = random.choice(p_item_ids)
        except IndexError:
            positive_id = 0

        p_ids.append(positive_id)

        # pick one of the negative ids
        try:
            negative_id = random.choice(n_item_ids)
        except IndexError:
            negative_id = 0
        
        n_ids.append(negative_id)

        # define triplet
        triplets[0][i,:] = X_usr[anchor_id-1][:]
        
        if positive_id == 0:
            triplets[1][i,:] = np.zeros((n_item_features,))
        else:
            triplets[1][i,:] = X_item[positive_id-1][:]
        
        if negative_id == 0:
            triplets[2][i,:] = np.zeros((n_item_features,))
        else:
            triplets[2][i,:] = X_item[negative_id-1][:]

    if return_cache:
        cache = {'users': user_ids, 'positive': p_ids, 'negative': n_ids}
        return triplets, cache
    
    return triplets

In [ ]:
def get_triplets_hard(batch_size, X_usr, X_item, df, return_cache = False):
    # constant values
    n_user_features = X_usr.shape[1]
    n_item_features = X_item.shape[1]

    # define user_list
    user_list = list(df.index.values)

    # initialise result
    triplets = [np.zeros((batch_size, n_user_features)), # anchor
                np.zeros((batch_size, n_item_features)), # pos
                np.zeros((batch_size, n_item_features))  # neg
                ]
    user_ids = []
    p_ids = []
    n_ids = []
    
    for i in range(batch_size):
        # pick one random user for anchor
        anchor_id = random.choice(user_list)
        user_ids.append(anchor_id) 
        
        # all possible positive/negative samples for selected anchor
        p_item_ids = get_pos(df, anchor_id)
        n_item_ids = get_neg(df, anchor_id)
        
        # pick one of the positve ids
        try:
            positive_id = random.choice(p_item_ids)
        except IndexError:
            positive_id = 0

        p_ids.append(positive_id)
        
        # pick the most similar negative id
        try:
            n_min = np.argmin([(cosine_dist(X_item[positive_id-1], X_item[k-1])) for k in n_item_ids])
            negative_id = n_item_ids[n_min]
        except:
            try:
                negative_id = random.choice(n_item_ids)
            except IndexError:
                negative_id = 0
            
        n_ids.append(negative_id)
        
        # define triplet
        triplets[0][i,:] = X_usr[anchor_id-1][:]
        
        if positive_id == 0:
            triplets[1][i,:] = np.zeros((n_item_features,))
        else:
            triplets[1][i,:] = X_item[positive_id-1][:]
        
        if negative_id == 0:
            triplets[2][i,:] = np.zeros((n_item_features,))
        else:
            triplets[2][i,:] = X_item[negative_id-1][:]

    if return_cache:
        cache = {'users': user_ids, 'positive': p_ids, 'negative': n_ids}
        return triplets, cache
    
    return triplets

In [ ]:
get_triplets_hard(1, X_usr, X_item, df_matrix, return_cache = True)

In [ ]:
# Hyper parameters
evaluate_every = 100 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 10000 # No. of training iterations
n_val = 1000 # how many one-shot tasks to validate on

In [ ]:
import time
print("Starting ....")
t_start = time.time()

for i in range(1, n_iter+1):
    triplets = get_triplets_hard(batch_size, X_usr, X_item, df_matrix)
    loss = network_train.train_on_batch(triplets, None)
    n_iteration += 1
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("[{3}] Time for {0} iterations: {1:.1f} mins, Train Loss: {2}".format(i, (time.time()-t_start)/60.0,loss,n_iteration))

# serialize weights to HDF5
network_train.save_weights("network_train.h5")
network_predict.save_weights("network_predict.h5")
print("Saved model to disk")

In [ ]:
#Evolution
network_predict.predict([X_usr[:1], X_item[7:8]])

In [ ]:
network_predict.predict([X_usr[:1], X_item[:1]])

In [ ]:
import math

In [ ]:
def compute_probs(network,X_usr,X_item, thrs = 0.5):
    m = X_usr.shape[0]
    nbevaluation = int(m**2)
    probs = np.zeros((nbevaluation))
    y = np.zeros((nbevaluation))
    
    #For each user and item of our dataset
    k = 0
    for i in range(1,m):
        for j in range(1,m):
            probs[k] = float(network.predict([X_usr[i-1:i], X_item[j-1:j]]))
            if probs[k] > thrs:
                y[k] = 1
            else:
                y[k] = 0    
    return probs,y
#probs,yprobs = compute_probs(network,x_test_origin[:10,:,:,:],y_test_origin[:10])

def compute_metrics(probs,yprobs):
    # calculate AUC
    auc = roc_auc_score(yprobs, probs)
    # calculate roc curve
    fpr, tpr, thresholds = roc_curve(yprobs, probs)
    
    return fpr, tpr, thresholds,auc

def compute_interdist(network):
    res = np.zeros((nb_classes,nb_classes))
    
    ref_images = np.zeros((nb_classes,img_rows,img_cols,1))
    
    #generates embeddings for reference images
    for i in range(nb_classes):
        ref_images[i,:,:,:] = dataset_test[i][0,:,:,:]
    ref_embeddings = network.predict(ref_images)
    
    for i in range(nb_classes):
        for j in range(nb_classes):
            res[i,j] = dist(ref_embeddings[i],ref_embeddings[j])
    return res

def draw_interdist(network,n_iteration):
    interdist = compute_interdist(network)
    
    data = []
    for i in range(nb_classes):
        data.append(np.delete(interdist[i,:],[i]))

    fig, ax = plt.subplots()
    ax.set_title('Evaluating embeddings distance from each other after {0} iterations'.format(n_iteration))
    ax.set_ylim([0,3])
    plt.xlabel('Classes')
    plt.ylabel('Distance')
    ax.boxplot(data,showfliers=False,showbox=True)
    locs, labels = plt.xticks()
    plt.xticks(locs,np.arange(nb_classes))

    plt.show()
    
def find_nearest(array,value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1],idx-1
    else:
        return array[idx],idx
    
def draw_roc(fpr, tpr,thresholds):
    #find threshold
    targetfpr=1e-3
    _, idx = find_nearest(fpr,targetfpr)
    threshold = thresholds[idx]
    recall = tpr[idx]
    
    
    # plot no skill
    plt.plot([0, 1], [0, 1], linestyle='--')
    # plot the roc curve for the model
    plt.plot(fpr, tpr, marker='.')
    plt.title('AUC: {0:.3f}\nSensitivity : {2:.1%} @FPR={1:.0e}\nThreshold={3})'.format(auc,targetfpr,recall,abs(threshold) ))
    # show the plot
    plt.show()

In [ ]:
#Testing
probs,yprob = compute_probs(network_predict,X_usr[:100],X_item[:100], thrs=0.3)
fpr, tpr, thresholds,auc = compute_metrics(probs,yprob)
draw_roc(fpr, tpr,thresholds)